In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig

# Define the replicator dynamics equations
def F_beta(beta, alpha):
    return beta(1- beta) * (b - d - b*alpha + v*b*alpha + v*w*alpha)

def F_alpha(alpha, beta):
    return alpha(1 - alpha) * (-c + a - m*p - a*v*beta - v*n*s*beta + v*m*p*beta)

# Jacobian matrix to determine stability
def jacobian(beta, alpha):
    dF_beta_dalpha = beta * (1 - beta) * (-b + v*b + v*w)
    dF_alpha_dbeta = alpha * (1 - alpha) * (-a*v - v*n*s + v*m*p)
    dF_beta_dbeta = (1 - 2 * beta) * (b - d + b*alpha - v*b*alpha - v*w*alpha)
    dF_alpha_dalpha = (2 * alpha - 1) * (c - a + m*p + a*v*beta + v*n*s*beta - v*m*p*beta)
    return np.array([[dF_beta_dbeta, dF_beta_dalpha], [dF_alpha_dbeta, dF_alpha_dalpha]])

# Helper function to classify equilibrium stability
def classify_stability(eigenvalues):
    if np.all(np.real(eigenvalues) < 0):
        return "Stable"
    elif np.all(np.real(eigenvalues) > 0):
        return "Unstable"
    else:
        return "Saddle"

# Maximum number of iterations for simulation
max_iterations = 100000
iteration = 0

# Set fixed values for some parameters
m, p, n, s = 0, 0, 0, 0  # Fixed to zero

# Initialize the best configuration and stability tracking
max_stable_count = 0
best_parameters = None
best_stability_ratio = 0

# Loop through maximum iterations to find the best configuration

while iteration < max_iterations:
    # Randomly generate parameter values
    w = np.round(np.random.uniform(0, 1), 2)  # a < w
    c = np.round(np.random.uniform(0, 1), 2)  # 0 < c < 1
    a = np.round(np.random.uniform(c, 2*w), 2)# c < a
    d = np.round(np.random.uniform(0, 1), 2)  # 0 < d < 1
    b = np.round(np.random.uniform(d, w), 2)  # d < b < w
    v = np.round(np.random.uniform(0, 1), 2)  # 0 ≤ v ≤ 1
    m = 0
    n = 0
    s = 0
    p = 0

    # Define equilibrium points (corner points)
    equilibrium_points = [(0, 0), (0, 1), (1, 0), (1, 1)]

    # Compute the internal equilibrium point based on given formulas
    if (v*b - b + v*w) == 0:
      pass
    else:
      beta_internal = round((d - b) / (v*b - b + v*w), 2)
    if v*(a - m*p + n*s)==0 :
      pass
    else:
      alpha_internal = round((a - c - m*p) / (v*(a - m*p + n*s)),2)

    # Check if internal equilibrium is within the [0, 1] range
    if 0 <= alpha_internal <= 1 and 0 <= beta_internal <= 1:
        equilibrium_points.append((beta_internal, alpha_internal))

    # Track stability of equilibrium points
    stable_count = 0
    for (beta_eq, alpha_eq) in equilibrium_points:
        J = jacobian(beta_eq, alpha_eq)
        eigenvalues = eig(J)[0]
        stability = classify_stability(eigenvalues)

        # Count the number of stable equilibria
        if stability == "Stable":
            stable_count += 1

    # Update the best configuration if the current one is better
    stability_ratio = stable_count / len(equilibrium_points)
    if stable_count > max_stable_count:
        max_stable_count = stable_count
        best_stability_ratio = stability_ratio
        best_parameters = {
            "a": a, "b": b, "c": c, "d": d, "w": w,
            "v": v, "stable_count": stable_count,
            "total_points": len(equilibrium_points)
        }

    # Increment iteration count
    iteration += 1

# Display the best configuration and its stability ratio
if best_parameters:
    print(f"Best configuration found after {iteration} iterations:")
    print(f"Parameters: a={best_parameters['a']:.4f}, b={best_parameters['b']:.4f}, c={best_parameters['c']:.4f}, d={best_parameters['d']:.4f}, w={best_parameters['w']:.4f}")
    print(f"v={best_parameters['v']:.4f}")
    print(f"Stable Equilibrium Points: {best_parameters['stable_count']} out of {best_parameters['total_points']}")
    print(f"Stability Ratio: {best_stability_ratio:.2%}")
else:
    print("No stable equilibria found within the iteration limit.")


Best configuration found after 100000 iterations:
Parameters: a=1.0100, b=0.8800, c=0.7000, d=0.8600, w=0.9000
v=0.6400
Stable Equilibrium Points: 2 out of 4
Stability Ratio: 50.00%


In [4]:
# Code to find Eigen values

import sympy as sp

# Define symbols
beta, alpha = sp.symbols('beta alpha')
b, d, v, w = sp.symbols('b d v w')
a, c, m, p, n, s = sp.symbols('a c m p n s')

# Jacobian matrix
J = sp.Matrix([
    [(1 - 2*beta) * (b - d - b*alpha + v*b*alpha + v*w*alpha), beta * (1 - beta) * (-b + v*b + v*w)],
    [alpha * (1 - alpha) * (-a*v - v*n*s + v*m*p), (1 - 2*alpha) * (-c + a - m*p - a*v*beta - v*n*s*beta + v*m*p*beta)]
])

# Define points to evaluate
points = [(0, 0), (0, 1), (1, 0), (1, 1),((a-c-m*p)/(v*(a-m*p+n*s)),(d-b)/(v*b-b+v*w))]

# Calculate and print symbolic eigenvalues for each point
for (beta_val, alpha_val) in points:
    J_eval = J.subs({beta: beta_val, alpha: alpha_val})
    eigenvals = J_eval.eigenvals()
    print(f"Eigenvalues for (beta, alpha) = ({beta_val}, {alpha_val}): {eigenvals}")


Eigenvalues for (beta, alpha) = (0, 0): {b - d: 1, a - c - m*p: 1}
Eigenvalues for (beta, alpha) = (0, 1): {b*v - d + v*w: 1, -a + c + m*p: 1}
Eigenvalues for (beta, alpha) = (1, 0): {-b + d: 1, -a*v + a - c + m*p*v - m*p - n*s*v: 1}
Eigenvalues for (beta, alpha) = (1, 1): {-b*v + d - v*w: 1, a*v - a + c - m*p*v + m*p + n*s*v: 1}
Eigenvalues for (beta, alpha) = ((a - c - m*p)/(v*(a - m*p + n*s)), (-b + d)/(b*v - b + v*w)): {-sqrt(-(b - d)*(-a + c + m*p)*(b*v - d + v*w)*(-a*v + a - c + m*p*v - m*p - n*s*v)/(v*(-a + m*p - n*s)*(b*v - b + v*w))): 1, sqrt(-(b - d)*(-a + c + m*p)*(b*v - d + v*w)*(-a*v + a - c + m*p*v - m*p - n*s*v)/(v*(-a + m*p - n*s)*(b*v - b + v*w))): 1}
